# Data Preparation

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 

C:\Users\Antonio\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
current_path = os.getcwd()

In [3]:
data_set = pd.read_csv(current_path + "\\Dataset\\pre_dataset_rq2.csv", low_memory=False) 

In [4]:
df = data_set.copy()

In [5]:
data_set.shape

(384, 30)

In [6]:
data_set.head(3)

,#,Jurisdiction,Kind,Publication_Number,Lens_ID,Publication_Date,Publication_Year,Application_Number,Application_Date,Priority_Numbers,...,Extended_Family_Size,Sequence_Count,CPC_Classifications,IPCR_Classifications,US_Classifications,NPL_Citation_Count,NPL_Resolved_Citation_Count,NPL_Resolved_Lens_ID(s),NPL_Resolved_External_Id(s),NPL_Citations
0,1,US,A1,US 2015/0068116 A1,193-193-445-785-000,2015-03-12,2015,US 201313987853 A,2013-09-09,US 201313987853 A 20130909,...,1,0,A01G13/0206,A01G17/04;;B62D49/00,47/44;;280/29;;800/298,0,0,NaN,NaN,NaN
1,4,WO,A1,WO 2016/187002 A1,046-633-029-572-58X,2016-11-24,2016,US 2016/0032331 W,2016-05-13,US 201562162430 P 20150515,...,10,0,NaN,A61B34/10;;A61B34/30;;A61B34/35;;B25J9/16,NaN,1,0,NaN,NaN,None
2,6,AU,A1,AU 2016/263106 A1,036-662-320-870-232,2017-12-14,2017,AU 2016/263106 A,2016-05-13,US 201562162430 P 20150515,...,10,0,NaN,A61B34/10;;A61B34/30;;A61B34/35;;B25J9/16,NaN,0,0,NaN,NaN,NaN


In [7]:
data_set.isnull().sum()

#                                0
Jurisdiction                     0
Kind                             0
Publication_Number               0
Lens_ID                          0
Publication_Date                 0
Publication_Year                 0
Application_Number               0
Application_Date                 0
Priority_Numbers                 0
Earliest_Priority_Date           0
Title                            0
Applicants                       0
Inventors                        0
Owners_(US)                    261
URL                              0
Type                             0
Has_Full_Text                    0
Cited_by_Patent_Count            0
Simple_Family_Size               0
Extended_Family_Size             0
Sequence_Count                   0
CPC_Classifications            162
IPCR_Classifications             1
US_Classifications             282
NPL_Citation_Count               0
NPL_Resolved_Citation_Count      0
NPL_Resolved_Lens_ID(s)        332
NPL_Resolved_Externa

In [8]:
data_set = data_set.dropna(subset=['IPCR_Classifications'])
data_set = data_set.dropna(subset=['Applicants'])
data_set = data_set.reset_index(drop=True)

In [9]:
columns = ['#', 'Jurisdiction', 'Kind', 'Lens_ID',
       'Publication_Date', 'Application_Number',
       'Application_Date', 'Priority_Numbers', 'Earliest_Priority_Date',
       'Title', 'Inventors', 'Owners_(US)', 'URL', 'Type',
       'Has_Full_Text', 'Cited_by_Patent_Count', 'Simple_Family_Size',
       'Extended_Family_Size', 'Sequence_Count', 'CPC_Classifications',
       'US_Classifications', 'NPL_Citation_Count',
       'NPL_Resolved_Citation_Count', 'NPL_Resolved_Lens_ID(s)',
       'NPL_Resolved_External_Id(s)', 'NPL_Citations']

In [10]:
data_set.drop(columns, axis=1, inplace=True)

In [11]:
data_set.shape

(383, 4)

#### Preparation column fro applicants categorization

In [12]:
for i in range(len(df)):
    if df.loc[i, 'Applicants'] == df.loc[i, 'Inventors']:
        df.at[i, 'Applicants'] = 'Inventors as applicant'

In [13]:
choices = df.Applicants.unique()
match = process.extract('UNIV', choices, limit=100, scorer=fuzz.token_sort_ratio)
for m in match:
    if m[1]>40:
        df.Applicants.replace(m[0],'UNIVERSITY', inplace = True)

In [14]:
for i in range(len(df)):
    if df.loc[i, 'Applicants'] != 'UNIVERSITY' and df.loc[i, 'Applicants'] != 'Inventors as applicant':
        df.at[i, 'Applicants'] = 'FIRMS'

In [15]:
data_set["Cat_Applicants"] = df.Applicants

#### Fuzzy wuzzy on assignees

In [16]:
choices = data_set.Applicants.unique()
for app in list(data_set.Applicants.value_counts().index[:28]):
    match = process.extract(app, choices, limit=100, scorer=fuzz.token_sort_ratio)
    for m in match:
        if m[1]>75:
            data_set.Applicants.replace(m[0], app, inplace = True)

In [17]:
data_set.to_csv(current_path+"\\Dataset\\dataset_rq2.csv", index=False)